In [273]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
from math import log
import csv
import string 

In [413]:
#Portfolio Extraction

data = pd.DataFrame(columns=['Ticker','Market Cap','Benchmark Proportion', 'Returns'])
ticker_name= []
#Reads ticker_name.csv: Column of Ticker Name, Column of Expexted Investor Returns in %
with open("ticker_name.csv", errors="ignore") as f:
    reader = csv.reader(f, delimiter=",")
    for i in reader:
        if i != "":
            ticker_name.append(i[0]+".NS")

for i in range(len(ticker_name)):
    if ticker_name[i].strip() != ".NS":
        data.loc[i, "Ticker"]= ticker_name[i] 
for i in range(len(data['Ticker'].values)):
    if(data['Ticker'].values[i].lstrip() != ".NS"):
        ticker_data = yf.download(data['Ticker'].values[i], start=(datetime.today()+ relativedelta(days = -60)).strftime('%Y-%m-%d'), end=datetime.today().strftime('%Y-%m-%d'), interval='1d')
        ticker_info = yf.Ticker(data['Ticker'].values[i])
        try:
            data.loc[i, "Market Cap"] = ticker_info.info['marketCap']
        except Exception: 
            pass
        tick_data = str(list(ticker_data["Adj Close"]))
        data.at[i, "Returns"] = tick_data
for i in range(len(data)):
    data.loc[i, 'Benchmark Proportion'] = data.loc[i, 'Market Cap']/data['Market Cap'].sum()
        
data.to_excel('data.xlsx')
#After execution of this block, 
# 1. Open data.xlsx and delete the index row.
# 2. Use text to columns to seperate the list of close prices to different cells.
# 3. Remove all [] brackets using CTRL+H
# 4. Raname Returns column as Price 1, and then name all other prices as Price 2 to Price 43(currently)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

PermissionError: [Errno 13] Permission denied: 'data.xlsx'

In [469]:
#Variance-Covariance Matric Calculation


price_data = pd.read_excel("data.xlsx")
col_size = price_data.shape[1]
row_size = price_data.shape[0]
returns = []
#Calculates log returns into returns
for i in range(0, row_size):
    entry = []
    for j in range(3, col_size-1):
        num = price_data.loc[i, 'Price ' + str(j-2)]
        denom = price_data.loc[i, 'Price ' + str(j-1)]
        entry.append((log(num/denom)))
    returns.append(entry)

varcov = np.cov(returns)
inverse = np.linalg.inv(varcov)

In [417]:
#Normalizing Factor Calculation
prop = data['Benchmark Proportion']
bench_prop = np.matrix(prop)   #Benchmark Proportions, Market share of stock i in the portfolio
a_ret = 1     #Anticipated returns 
rfr = 0.127   #Risk Free Rate
benchprop_t = bench_prop.transpose()

norm_fact = (a_ret-rfr)/(np.dot(bench_prop, np.dot(varcov, benchprop_t)))
print(norm_fact)

[[7530.42556891021]]


In [419]:
#Expected Benchmark Return with no investor opinion
exp_benchret_no = np.dot(varcov, benchprop_t)*norm_fact+rfr
print(exp_benchret_no)

[[0.88521856180566]
 [0.8600531572790198]
 [0.6030957359359475]
 [0.8005749402527231]
 [0.6549419645193454]
 [1.128655990768649]
 [0.9480162812791308]
 [0.6877181903599972]
 [0.7743841136052871]
 [1.1919101194555448]
 [1.1587065146810003]
 [1.008350004904983]
 [0.744760685585209]
 [0.819956676601485]
 [0.844927766275275]
 [0.6965480955717861]
 [0.31836594574893906]
 [1.2309646005778878]
 [0.9447109094533371]
 [0.39396267531594165]]


In [421]:
#optimized Portfolio with no investor opinion
opt_port_no = np.dot(inverse,exp_benchret_no-rfr)/(np.dot(inverse,exp_benchret_no-rfr)).sum()
print(opt_port_no)

[[0.004995683901439752]
 [0.17737346966637121]
 [0.01340164158797769]
 [0.0038806352193226142]
 [0.036991416511071566]
 [0.6290602278805622]
 [0.00884846135903415]
 [0.001358415129901061]
 [0.0005257757218774887]
 [0.0026533931387407966]
 [0.005400159217540468]
 [0.006084078695618569]
 [0.004803578338538389]
 [0.028995367461380556]
 [0.007492152702714615]
 [0.013401413035288781]
 [0.014344266210354395]
 [0.006874949221243146]
 [0.0055975804637753345]
 [0.027917334537247148]]


In [465]:
# Expected Benchmark Returns with Investor Opinion
blm_matrix = np.matrix(varcov/np.var(returns))
inv_opinion = []
with open("ticker_name.csv", errors="ignore") as f:
    reader = csv.reader(f, delimiter=",")
    for i in reader:
        if i != "":
            inv_opinion.append(float(i[1]))
#inv_op_matrix = np.matrix(inv_opinion, data = 'float')
opinion_returns = exp_benchret_no + np.dot(inv_opinion, blm_matrix).transpose()
print(opinion_returns)

[[1.7999996807924683]
 [1.4321318955437619]
 [1.4508545801226418]
 [1.8922866520170964]
 [1.6009941555056502]
 [1.4831525686968416]
 [1.6163315040659398]
 [1.4366819847520758]
 [1.6144583256259897]
 [2.4606594803823816]
 [2.1606719300170645]
 [2.0311731814652005]
 [1.6663787929615173]
 [1.5409720427833844]
 [1.7931135688319477]
 [1.1430089254570037]
 [0.8549435657353358]
 [2.487496979305008]
 [1.9060829735513685]
 [0.514153614982875]]


In [467]:
#Optimized Portfolio adjusted for Investor Opinion
opt_port_weights = np.dot(inverse,opinion_returns-rfr)/(np.dot(inverse,opinion_returns-rfr)).sum()
print(opt_port_weights)

[[0.004733630162652068]
 [0.17218073241071616]
 [0.008277243435289114]
 [0.002396793122918963]
 [0.09536559877783217]
 [0.3967666365406744]
 [0.00876136808088288]
 [0.018968652820538674]
 [0.021750691634595778]
 [0.023064770001856295]
 [0.024761252581603263]
 [0.02518366113305025]
 [0.014503883535168295]
 [0.032741736879050874]
 [0.009571822985148417]
 [0.01816600562724632]
 [0.018748339157248488]
 [0.05533883581879437]
 [0.023235035123594667]
 [0.02548331017113863]]
